<a href="https://colab.research.google.com/github/Manish533005/nlp_chatbot/blob/main/training_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
from tensorflow import keras
import json
import spacy
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [ ]:
# loading json  data into a file  and converting it into dictionary  
data_file=open("intents.json").read()
intents=json.loads(data_file)

In [ ]:
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day'], 'responses': ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'options', 'patterns': ['How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered'], 'responses': ['I can guide you through

In [ ]:
type(intents)

dict

In [ ]:
nlp=spacy.load("en_core_web_sm")
words =[]
classes=[]
documents=[]
ignore_words=['?','!']


In [ ]:
words=[]
for intent in intents['intents']:
    for pattern in intent['patterns']:
        pattern=str(pattern)
        w=[]
        #tokenize each word
        for word in pattern.split():
          if word != '.':
             word = word.replace('.','')
             w.append(word)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
print(w)
print(words)
print(len(words))
print(documents)
print(classes)

['Looking', 'up', 'hospital', 'details']
['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there?', 'Hey', 'Hola', 'Hello', 'Good', 'day', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you,', 'bye', 'Till', 'next', 'time', 'Thanks', 'Thank', 'you', "That's", 'helpful', 'Awesome,', 'thanks', 'Thanks', 'for', 'helping', 'me', 'How', 'you', 'could', 'help', 'me?', 'What', 'you', 'can', 'do?', 'What', 'help', 'you', 'provide?', 'How', 'you', 'can', 'be', 'helpful?', 'What', 'support', 'is', 'offered', 'How', 'to', 'check', 'Adverse', 'drug', 'reaction?', 'Open', 'adverse', 'drugs', 'module', 'Give', 'me', 'a', 'list', 'of', 'drugs', 'causing', 'adverse', 'behavior', 'List', 'all', 'drugs', 'suitable', 'for', 'patient', 'with', 'adverse', 'reaction', 'Which', 'drugs', 'dont', 'have', 'adverse', 'reaction?', 'Open', 'blood', 'pressure', 'module', 'Task', 'related', 'to', 'blood', 'pressure', 'Blood', 'pressure', 'data', 'entry', 'I', 'want', 'to', 'log', 'blood', 'p

In [ ]:
text="hi my name is Manish"
tokens=nlp(text)
l=[]
for i in tokens:
    i=str(i)
    l.append(i)
print(l)

['hi', 'my', 'name', 'is', 'Manish']


In [ ]:
type(tokens)

spacy.tokens.doc.Doc

In [ ]:
words1=[]
documents1=[]
classes1=[]
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        tokens= nlp(pattern)
        l=[]
        for i in tokens:
             i=str(i)
             l.append(i)
        words1.extend(l)
        #add documents in the corpus
        documents1.append((l, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes1:
            classes1.append(intent['tag'])

In [ ]:
print(l)
print(words1)
print(len(words1))
print(documents1)
print(classes1)

['Looking', 'up', 'hospital', 'details']
['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello', 'Good', 'day', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'Till', 'next', 'time', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Awesome', ',', 'thanks', 'Thanks', 'for', 'helping', 'me', 'How', 'you', 'could', 'help', 'me', '?', 'What', 'you', 'can', 'do', '?', 'What', 'help', 'you', 'provide', '?', 'How', 'you', 'can', 'be', 'helpful', '?', 'What', 'support', 'is', 'offered', 'How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?', 'Open', 'adverse', 'drugs', 'module', 'Give', 'me', 'a', 'list', 'of', 'drugs', 'causing', 'adverse', 'behavior', 'List', 'all', 'drugs', 'suitable', 'for', 'patient', 'with', 'adverse', 'reaction', 'Which', 'drugs', 'do', 'nt', 'have', 'adverse', 'reaction', '?', 'Open', 'blood', 'pressure', 'module', 'Task', 'related', 'to', 'blood', 'pressure', 'Blood', 'pressure', 'data', 'e

In [ ]:
training= []

output_empty = [0] * len(classes)

for doc in documents1:
    
    bag=[]

    pattern_words=doc[0]

    pattern_words=[     for word in pattern_words]

